In [189]:
from pathlib import Path
import shutil

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import locale
#from html.parser import HTMLParser
# import website_func.py to use its functions
from website_func import *
import os #To read the file
#reload every module each time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Some quick analysis of the data
We received a data set of .html file containing the website content of recipes.
We wanted to sort them by website, in order to, more easily, find a pattern among them. This will enable us to do the "scraping" of the pages. First we thought about moving the files in a folder corresponding to their website, but it would be a waste of time and a big computational effort. Thus, we came up with a (probably) faster solution : we could simply write the name of the file within its corresponding website folder. By inspecting the files, we saw that the first line was always containing a comment with the name of the file and the complete website. Using readlines and split, we could easily retrieve the name of the website.

We launched this process, but an alarm appeared describing a Trojan virus in the file "1c2cb6f0df04cf5a9d0baa116c6aa7bb.html". 
We had then to quarantine or mabye remove the file, as we have quite enough
By doing so, we remarked the file "msg.log" that could help us into fastering the processus as its content is formed of the name of the file together with its website. [SHOW] We obserbed that occasionally a line containing other info that the content we want can appear. We need to ignore this
Also we noticed there are no file extensions other than .html and .log. 

In [14]:
retrieve_website_from_log()

FileNotFoundError: [Errno 2] No such file or directory: 'recipePages.zip\\recipePages\\msg.log'

In [16]:
# We can find in the folder that, excepted the html files, there is only the log

# Get all filenames (i.e. path) that are in recipePages folder 
pathlist = Path("recipePages/").glob('**/*')
i = 0
for path in pathlist:
     # because path is object not string
    path_in_str = str(path)
    if not path_in_str.endswith(".html"):
        print(path_in_str)

recipePages\msg.log


In [87]:
website_count=pd.Series()

for (root,dirs,files) in os.walk('SortedFiles', topdown="True"):   
        for website in dirs:
            
            filename = "SortedFiles/" + website +"/filesName.txt"
            
            f = open(filename, "r")
            number_of_link = len(f.readlines())
            line_to_add=pd.Series(number_of_link,index=[website])
            website_count = website_count.append(line_to_add)

In [88]:
website_count.sort_values(ascending=False,inplace=True)

In [118]:
website_count_used=website_count[website_count.values>1000]

In [138]:
website_list_used=website_count_used.index.tolist()

['allrecipes.com', 'food.com', 'foodnetwork.com', 'yummly.com', 'cooks.com', 'epicurious.com', 'tasteofhome.com', 'myrecipes.com', 'recipes.sparkpeople.com', 'cdkitchen.com', 'bettycrocker.com', 'cookeatshare.com', 'southernfood.about.com', 'grouprecipes.com', 'recipe.com', 'kraftrecipes.com', 'eatingwell.com', 'ifood.tv', 'delish.com']


In [256]:

for (root,dirs,files) in os.walk('recipePagesT', topdown="True"):  
    for webpage in files:
        filename= 'recipePagesT/'+webpage
        
        #Try to open it in a simple way to avoid expensive calculation
        try:
            f = open(filename,'r')
            first_line=f.readline()
            f.close()
        except:
            continue
            #If we can't open it in a simple way go do it with Beautifulsoup
            #try:
            #    tree = html.parse(filename)
            #    website = html.tostring(tree)  
            #    i=1;
            #except:
            #    print("We aren't able to read a document called:" + wesite)
            #    continue
        website = str(first_line).split("/")[2].strip("www.")
        
        #Determine if we have to spare the data
        if website in website_list_used :
            
            #Read the data from the first website
            if website in website_list_used[0]:
                print(filename)
                soup = BeautifulSoup(open(filename), "html.parser")
                
                recipe_name = soup.find('span', class_='itemreviewed')
                
                #We only take recipe and not searching pages
                if recipe_name is None:
                    print("We don't care about this page")
                    continue
                recipe_name = recipe_name.text
                rating_html = soup.find('img', class_='rating')
                
                #Determine the postion
                start_rank = 59
                end_rank = 62
                rating = rating_html['title'][start_rank:end_rank]
                
                print(recipe_name)
                print(rating)
                
                #Determine the number of reviews
                review_html = soup.find('span', class_='count').text
                review = int(review_html.replace(',',''))
                print('Reviews :',review)
                
                #Time to prepare
                prepare_time = np.nan
                soup1 = soup.find('span', class_='totalTime')
                if soup1 is not None:
                    
                    prepare_time_html = soup1.find('span', class_='value-title')
                    #Determine the postion
                    start_prepare_time = 2
                    end_prepare_time = len(prepare_time_html['title']) 
                    prepare_time_not_converted = prepare_time_html['title'][start_prepare_time:end_prepare_time]
                    time_analyse = prepare_time_not_converted.split('H')
                    
                    #We only have minutes
                    if len(time_analyse) == 1:
                        prepare_time = int(time_analyse[0].replace('M',''))
                    if len(time_analyse) == 2:
                        time_minute = time_analyse[1].replace('M','');
                        prepare_time = int(time_analyse[0])*60 + (int(time_minute) if time_minute else 0) 
                    
                    
                
                print(type(prepare_time))
                print('\n')

recipePagesT/000a3333ad24828769b6be5a5e1bdb4a.html
Chicken Breast Cutlets with Artichokes and Capers
4.4
Reviews : 80
<class 'int'>


recipePagesT/000b861ad15679c578d81884a87689ea.html
Best Ever Popcorn Balls
4.4
Reviews : 322
<class 'int'>


recipePagesT/00a149d30ee1176043ed750ac51b4fee.html
Orange Cream Cheese Frosting
4.6
Reviews : 39
<class 'float'>


recipePagesT/00a1d99a6383cdeac3d6c253f8e7cff1.html
We don't care about this page
recipePagesT/00a405ea8c3d491b677a995cb558b99f.html
Perfect Baked Potato
4.7
Reviews : 410
<class 'int'>


recipePagesT/00a50320c5b99874292e760601993c21.html
Pumpkin Oatmeal
3.9
Reviews : 88
<class 'int'>


recipePagesT/00a53bf43b5779e20f2ec5bfca29dda4.html
Baked Asparagus with Balsamic Butter Sauce
4.6
Reviews : 1319
<class 'int'>


recipePagesT/00a58140b06973b9ac0da04b4b10f67e.html
Chicken Corn Chowder
4.5
Reviews : 42
<class 'int'>


recipePagesT/00a7450a98369de9097e042172e59fa4.html
We don't care about this page
recipePagesT/00ab5078eaeeb0bcec9284b8dad

In [222]:
print(soup1)

None
